In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import warnings
warnings.filterwarnings('ignore')

from selenium.common.exceptions import WebDriverException

In [8]:
# scrape images 
url = 'https://www.nba.com/standings'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)
# find by class MaxWidthContainer_mwc__ID5AG
main = driver.find_element_by_class_name('MaxWidthContainer_mwc__ID5AG')
# find all images
images = main.find_elements_by_tag_name('img')
# get src
src = [image.get_attribute('src') for image in images]
src

['https://cdn.nba.com/logos/nba/1610612738/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612751/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612749/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612739/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612755/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612754/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612752/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612748/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612741/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612737/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612764/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612761/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612753/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612766/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/1610612765/primary/L/logo.svg',
 'https://cdn.nba.com/logos/nba/16106127

In [13]:
len(src)

30

In [18]:
# download logos
#os.mkdir('data/team/logos')
#os.chdir('data/team/logos')
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
driver = webdriver.Chrome()
for pic in src:
    driver.get(pic)
    # download image, save to data/team/logos
    driver.save_screenshot('data/team/logos/'+pic.split('/')[-4]+ '.png')

In [63]:
url = 'https://www.nba.com/players'
driver = webdriver.Chrome()
driver.get(url)

all_xpath = '//*[@id="__next"]/div[2]/div[2]/main/div[2]/section/div/div[2]/div[1]/div[7]/div/div[3]/div/label/div/select/option[1]'
# wait for xpath to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, all_xpath)))
all_players = driver.find_element_by_xpath(all_xpath)
all_players.click()

# wait 5 seconds
time.sleep(5)
# this loads all pages
# Now, for each player, get their image url and name
soup = BeautifulSoup(driver.page_source, 'html.parser')

tables = soup.find('table', class_ = 'players-list')
tbody = tables.find('tbody')
# get all rows
rows = tbody.find_all('tr')

# get all images
images = [row.find('img').get('src') for row in rows]
# get all names
names = [row.find('a').get_text() for row in rows]
# get htmls
htmls = [row.find('a').get('href') for row in rows]

# get df
df = pd.DataFrame({'name': names, 'image': images, 'html': htmls})
df.head()

,name,image,html
0,PreciousAchiuwa,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630173/precious-achiuwa/
1,StevenAdams,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/203500/steven-adams/
2,BamAdebayo,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1628389/bam-adebayo/
3,OchaiAgbaji,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630534/ochai-agbaji/
4,SantiAldama,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630583/santi-aldama/


In [64]:
df

,name,image,html
0,PreciousAchiuwa,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630173/precious-achiuwa/
1,StevenAdams,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/203500/steven-adams/
2,BamAdebayo,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1628389/bam-adebayo/
3,OchaiAgbaji,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630534/ochai-agbaji/
4,SantiAldama,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630583/santi-aldama/
...,...,...,...
508,McKinleyWright IV,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630593/mckinley-wright-iv/
509,ThaddeusYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/201152/thaddeus-young/
510,TraeYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1629027/trae-young/
511,OmerYurtseven,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630209/omer-yurtseven/


In [65]:
df.to_csv('data/player/nba_com_info/players_and_photo_links.csv', index = False)

In [66]:
all_images = df['image'].to_list()

In [67]:
# download all images
os.chdir('data/player/nba_com_info/images')
driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\chromedriver.exe')
for pic in all_images:
    driver.get(pic)
    # download image, save to data/team/logos
    driver.save_screenshot(pic.split('/')[-1])

In [74]:
df['player_name'] = df['html'].str.split('/').str[-1].replace('-', ' ')
df

,name,image,html,player_name
0,PreciousAchiuwa,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630173/precious-achiuwa/,
1,StevenAdams,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/203500/steven-adams/,
2,BamAdebayo,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1628389/bam-adebayo/,
3,OchaiAgbaji,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630534/ochai-agbaji/,
4,SantiAldama,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630583/santi-aldama/,
...,...,...,...,...
508,McKinleyWright IV,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630593/mckinley-wright-iv/,
509,ThaddeusYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/201152/thaddeus-young/,
510,TraeYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1629027/trae-young/,
511,OmerYurtseven,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630209/omer-yurtseven/,


In [81]:
# str split on /, get last element, replace - with space
df['player_name'] = df['html'].str.split('/').str[3].str.replace('-', ' ')
df['nba_id'] = df['html'].str.split('/').str[2]

In [82]:
df

,name,image,html,player_name,nba_id
0,PreciousAchiuwa,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630173/precious-achiuwa/,precious achiuwa,1630173
1,StevenAdams,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/203500/steven-adams/,steven adams,203500
2,BamAdebayo,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1628389/bam-adebayo/,bam adebayo,1628389
3,OchaiAgbaji,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630534/ochai-agbaji/,ochai agbaji,1630534
4,SantiAldama,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630583/santi-aldama/,santi aldama,1630583
...,...,...,...,...,...
508,McKinleyWright IV,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630593/mckinley-wright-iv/,mckinley wright iv,1630593
509,ThaddeusYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/201152/thaddeus-young/,thaddeus young,201152
510,TraeYoung,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1629027/trae-young/,trae young,1629027
511,OmerYurtseven,https://cdn.nba.com/headshots/nba/latest/260x1...,/player/1630209/omer-yurtseven/,omer yurtseven,1630209


In [85]:
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')

In [86]:
df.to_csv('data/player/nba_com_info/players_and_photo_links.csv', index = False)